In [7]:
from prediction import toolbox
from prediction.toolbox import BaseFeatureExtractor
from os import path as osp
import numpy as np
from pandas import DataFrame as df
subjects_filename = osp.join('puzzles4.json')
subjects_data = toolbox.load_subjects_data_from_json_file(subjects_filename)

successfully loaded 4 subjects from puzzles4.json


In [8]:
#puzzles functions
def get_dfc(subjects_data):
    dfc = PuzzlesInstanceTimeCompletion().extract(subjects_data)
    return dfc
def get_dfg(subjects_data):
    dfg = PuzzlesInstanceTimeGazePzlPostCompletion().extract(subjects_data)
    return dfg


In [9]:
#inserted
class PuzzlesInstanceTimeCompletion(BaseFeatureExtractor):
    """time until completion"""
    name = 'time_to_completion'
    column_prefix = 'i_tCompletion'
    @staticmethod
    def extract(subjects):
        def get_tCompletion(inst):
            sol,  last_swap = [], 0.0
            puzzle_start = inst['PuzzleStartTime']
            swap_l1 = []
            puzzle_swap = inst['PiecesSwapList']
            for pzl in puzzle_swap:
                pzl_time = pzl['Time']
                swap_l1.append([pzl_time])

            if swap_l1:
                for pz in swap_l1:
                    last_swap = pz[0]
                sol = last_swap - puzzle_start
                
            if not sol:
                sol = np.NAN
            
            return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleInstances', get_tCompletion , subjects)
        return df
df1 = PuzzlesInstanceTimeCompletion().extract(subjects_data)
df1

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
id,,,,,,,,,,,,,,,,,,,,,
5BB9D23220AB8A117956A91C,18.564392,5.265885,NaN,7.372758,37.229200,15.877105,4.116035,11.085632,1.547100,6.686828,...,6.813721,3.439727,26.690582,5.569362,3.943900,10.533642,7.57055,48.2759,19.3301,15.0894
5BBA2B3320AB8A117956A92D,7.103699,1.776001,7.080800,15.001529,2.959839,6.668934,1.798455,25.310881,11.203033,5.082045,...,8.600891,83.157500,12.432100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5BBB4EB94AC518563CE8D571,5.523193,1.642563,8.725472,4.227737,12.997326,19.034300,5.081645,1.284424,4.339172,5.306519,...,20.753072,14.391540,4.528778,89.080292,NaN,NaN,NaN,NaN,NaN,NaN
5BBB61C6DC69BC3FA50FA40A,11.337479,9.439011,1.642319,20.228775,9.103000,7.550537,5.684716,12.309814,4.668277,1.608398,...,4.266754,1.262751,17.156128,16.732330,5.930719,7.216100,58.85310,NaN,NaN,NaN


In [18]:
#inserted
class PuzzlesInstanceTimeLastSwapToNextSelection(BaseFeatureExtractor):
    """"""
    name = 'time_from_last_swap_to_next_selection'
    column_prefix = 'i_tLastSwapToNextSelection'
    @staticmethod
    def extract(subjects):
        def get_tPuzzleLastSwapToNextSelection(inst):
            sol, last_swap = [], 0.0
            puzzle_end = inst['PuzzleEndTime']
            swap_l1 = []
            puzzle_swap = inst['PiecesSwapList']
            for pzl in puzzle_swap:
                pzl_time = pzl['Time']
                pzl_uid = pzl['Piece1']['Uid']
                swap_l1.append([pzl_time, pzl_uid])

            if swap_l1 != []:
                for pz in swap_l1:
                    last_swap = pz[0]
                sol = puzzle_end - last_swap
            else:
                sol = np.NaN

            return sol
            sol.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles', 'PuzzleInstances', get_tPuzzleLastSwapToNextSelection , subjects)
        return df
PuzzlesInstanceTimeLastSwapToNextSelection().extract(subjects_data)


,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
id,,,,,,,,,,,,,,,,,,,,,
5C35A1D0B5997B32B4CEAA9B,2.6137,2.434875,2.368176,1.619903,1.820635,0.960425,3.474213,1.083282,1.7208,1.362721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
#inserted
class PuzzlesRndTimeCompletion(BaseFeatureExtractor):
    """Round time completion"""
    name = 'time_untill_completion'
    column_prefix = 'r_tCompletion'
    @staticmethod
    def extract(subjects):
        def get_tCompletion(rnd):
            sol, swap_l1, multi_in, gaze_l1, last_swap, agg =  [], [], [], [], 0.0, 0.0
            rnd_instances = rnd['PuzzleInstances']
            tot_puz_time = rnd['TotalPuzzleTime']
            for inst in rnd_instances:
                in_start_time = inst['PuzzleStartTime']
                in_end_time = inst['PuzzleStartTime']
                in_pzl_name = inst['CurrentPuzzle']['puzzleName']
                multi_in.append([in_pzl_name, in_start_time])

                pzl_swap_list = inst['PiecesSwapList']
                if pzl_swap_list != []:
                    for pzl in pzl_swap_list:
                        pzl_time = pzl['Time']
                        swap_l1.append([in_pzl_name, pzl_time, in_start_time, in_end_time])

                in_gaze_list = inst['ObjectsGazedAtList']
                if in_gaze_list != []:
                    for gaze in in_gaze_list:
                        gaze_len = gaze['length']
                        gaze_start = gaze['startTime']
                        gaze_name = gaze['name']
                        gaze_l1.append([gaze_start, gaze_name, gaze_len])

                if swap_l1 != '' :
                    for pz in swap_l1:
                        rnd_name = pz[0]
                        last_swap = pz[1]
                        if last_swap > in_start_time:
                            agg = last_swap - in_start_time
                        else:
                            for g in gaze_l1:
                                if g[0] > last_swap and ( g[1] == rnd_name or g[1][:5]=='Piece'):
                                    agg += g[2]
                    multi_in.append(['time', agg])

            for t1, t2 in zip(multi_in, multi_in[2:]):
                if t1[0] == t2[0]:
                    agg = t1[1] + t2[1]
            if agg > tot_puz_time:
                sol = tot_puz_time
            else:
                sol = agg
                
            return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleRounds', get_tCompletion , subjects)
        return df
PuzzlesRndTimeCompletion().extract(subjects_data)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
id,,,,,,,,,,,,,,,,,,,,,
5C35A1D0B5997B32B4CEAA9B,53.536682,6.590118,16.731873,13.694031,4.949997,12.823273,12.823029,18.87674,13.359741,6.668213,...,29.353943,13.38681,4.288757,8.019287,10.712585,13.962555,11.650421,12.734825,11.370697,13.600037


In [16]:
#not relevant
class PuzzlesInstanceTimeGazePzlPostCompletion(BaseFeatureExtractor):
    """Time while gazing on the same puzzle after completion"""
    name = 'time_gaze_pzl_post_completion'
    column_prefix = 'i_tGazePzlPostCompletion'
    @staticmethod
    def extract(subjects):
        def get_tGazePzlPostCompletion(inst):
            sol, swap_l1, gaze_l1, last_swap, agg = [], [], [], 0.0, 0.0
            start_time = inst['PuzzleStartTime']
            end_time = inst['PuzzleStartTime']
            pzl_name = inst['CurrentPuzzle']['puzzleName']
            pzl_swap_list = inst['PiecesSwapList']
            in_gaze_list = inst['ObjectsGazedAtList']
            if in_gaze_list != []:
                for gaze in in_gaze_list:
                    gaze_len = gaze['length']
                    gaze_start = gaze['startTime']
                    gaze_name = gaze['name']
                    gaze_l1.append([gaze_start, gaze_name, gaze_len])        
            if pzl_swap_list != []:
                for pzl in pzl_swap_list:
                    pzl_time = pzl['Time']
                    piece_uid = pzl['Piece1']['Uid']
                    swap_l1.append([pzl_name, pzl_time]) 
                if swap_l1 != '' :
                    last_swap = swap_l1[-1]
                for g in gaze_l1:
                    if g[0] > last_swap[1] and ( g[1] == last_swap[0] or g[1][:5]=='Piece'):
                        agg += g[2]
            if agg > 0:
                sol = agg
            else:
                sol = np.NaN

            return sol
            sol.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleInstances', get_tGazePzlPostCompletion , subjects)
        return df
df2 = PuzzlesInstanceTimeGazePzlPostCompletion().extract(subjects_data)
df2

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
id,,,,,,,,,,,,,,,,,,,,,
5BB9D23220AB8A117956A91C,0.456207,0.028748,NaN,0.462036,0.150696,0.346252,0.356995,0.016388,0.731567,0.886078,...,2.148682,0.915070,1.908966,0.893066,0.921753,1.853943,1.909668,0.458008,1.718567,NaN
5BBA2B3320AB8A117956A92D,0.837158,1.854340,0.837158,1.161438,0.792969,1.217682,1.373840,1.172760,1.317657,0.335022,...,9.495117,2.223450,0.614868,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5BBB4EB94AC518563CE8D571,0.837570,1.138275,0.881912,0.619431,1.776001,0.367126,2.636078,2.456818,0.815155,0.927521,...,1.507660,2.137695,0.964325,2.402313,NaN,NaN,NaN,NaN,NaN,NaN
5BBB61C6DC69BC3FA50FA40A,3.406250,3.485062,0.469849,0.268097,0.602325,0.010986,1.061707,2.657684,2.278290,0.915894,...,0.815277,1.318878,2.357544,2.736053,3.662140,1.977295,2.344788,NaN,NaN,NaN


In [23]:
#inserted
class PuzzlesRoundTimeGazePzlPostCompletion(BaseFeatureExtractor):
    """Time while gazing on the same puzzle after completion"""
    name = 'time_gaze_pzl_post_completion'
    column_prefix = 'r_tGazePzlPostCompletion'
    @staticmethod
    def extract(subjects):
        def get_tGazePzlPostCompletion(rnd):
            sol, swap_l1, gaze_l1, last_swap, agg = [], [], [], 0.0, 0.0
            for inst in rnd['PuzzleInstances']:
                pzl_name = inst['CurrentPuzzle']['puzzleName']
                pzl_swap_list = inst['PiecesSwapList']
                in_gaze_list = inst['ObjectsGazedAtList']
                if in_gaze_list != []:
                    for gaze in in_gaze_list:
                        gaze_len = gaze['length']
                        gaze_start = gaze['startTime']
                        gaze_name = gaze['name']
                        gaze_l1.append([gaze_start, gaze_name, gaze_len])        
                if pzl_swap_list != []:
                    for pzl in pzl_swap_list:
                        pzl_time = pzl['Time']
                        swap_l1.append([pzl_name, pzl_time]) 
                    if swap_l1 != '' :
                        last_swap = swap_l1[-1]
                    for g in gaze_l1:
                        if g[0] > last_swap[1] and ( g[1] == last_swap[0] or g[1][:5]=='Piece'):
                            agg += g[2]
                if agg > 0:
                    sol = agg
                else:
                    sol = np.NaN

                return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleRounds', get_tGazePzlPostCompletion, subjects)
        return df
PuzzlesRoundTimeGazePzlPostCompletion().extract(subjects_data)


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
id,,,,,,,,,,,,,,,,,,,,,
5C35A1D0B5997B32B4CEAA9B,NaN,0.4133,2.568481,0.312866,1.117096,0.782471,2.45874,2.02121,0.725494,1.083405,...,1.061646,2.345856,0.80426,2.603088,2.524597,0.815735,0.290253,1.619904,1.641541,3.127411


In [25]:
#Inserted
class PuzzlesInstancePuzzleGaze(BaseFeatureExtractor):
    """Total time gazing on a puzzle"""
    name = 'time_puzzle_gaze'
    column_prefix = 'i_puzzleGaze'
    @staticmethod
    def extract(subjects):
        def get_puzzleGaze(inst):
            sol, gaze_l1, agg = [], [], 0.0
            pzl_name = inst['CurrentPuzzle']['puzzleName']
            in_gaze_list = inst['ObjectsGazedAtList']
            if in_gaze_list != []:
                for gaze in in_gaze_list:
                    gaze_len = gaze['length']
                    gaze_start = gaze['startTime']
                    gaze_name = gaze['name']
                    gaze_l1.append([gaze_start, gaze_name, gaze_len])

                for g in gaze_l1:
                    if g[1] == pzl_name or g[1][:5]=='Piece':
                        agg += g[2]
            if agg > 0:
                sol = agg
            else:
                sol = np.NaN        
            return sol
            sol.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleInstances', get_puzzleGaze , subjects)
        return df
PuzzlesInstancePuzzleGaze().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
id,,,,,,,,,,,,,,,,,,,,,
5C35A1D0B5997B32B4CEAA9B,6.421722,3.082855,10.578796,8.958405,10.488129,10.968506,7.976837,7.683075,8.041626,2.245148,...,0.803894,1.765564,3.742065,1.876221,0.301575,0.64856,NaN,0.569641,0.413513,0.245667


In [9]:
#inserted
class PuzzlesRoundPuzzleGaze(BaseFeatureExtractor):
    """Total time gazing on a puzzle"""
    name = 'r_tpuzzleGaze'
    @staticmethod
    def extract(subjects):
        def get_puzzleGaze(rnd):
            sol, gaze_l1, agg = [], [], 0.0
            pzl_name = rnd['CurrentPuzzle']['puzzleName']
            for inst in rnd['PuzzleInstances']:
                in_gaze_list = inst['ObjectsGazedAtList']
                if in_gaze_list != []:
                    for gaze in in_gaze_list:
                        gaze_len = gaze['length']
                        gaze_start = gaze['startTime']
                        gaze_name = gaze['name']
                        gaze_l1.append([gaze_start, gaze_name, gaze_len])

                    for g in gaze_l1:
                        if g[1] == pzl_name or g[1][:5]=='Piece':
                            agg += g[2]
                if agg > 0:
                    sol = agg
                else:
                    sol = np.NaN        
                return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleRounds', get_puzzleGaze , subjects)
        return df
PuzzlesRoundPuzzleGaze().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
id,,,,,,,,,,,,,,,,,,,,,
5BB9D23220AB8A117956A91C,9.847855,9.293915,4.966705,8.847229,2.053955,13.348724,3.117218,6.301575,1.239807,0.914398,...,10.381958,10.437317,0.915070,3.183838,4.624969,0.637268,1.741180,0.658844,6.851868,17.448975
5BBA2B3320AB8A117956A92D,NaN,5.204193,33.117767,17.289215,3.552094,7.594818,5.438355,11.279541,17.290955,12.364990,...,9.840027,10.410584,14.765564,5.998260,3.963776,8.041870,21.838898,3.149506,7.784851,8.657776
5BBB4EB94AC518563CE8D571,NaN,5.412506,22.138733,22.306122,2.781616,6.299103,6.183044,19.091187,15.642059,4.774307,...,NaN,14.459839,2.741699,3.695587,6.506897,7.182617,4.891998,2.512848,6.075317,7.518219
5BBB61C6DC69BC3FA50FA40A,13.492035,5.663086,7.359802,7.852966,2.102722,14.385635,5.094208,8.654266,18.956848,5.160309,...,31.491119,17.253632,2.435547,2.558716,5.696716,13.663788,4.402008,2.525604,16.562378,19.178741


In [10]:
class PuzzlesInstanceTimeHeadPzlPostCompletion(BaseFeatureExtractor):
    """time head directed on the puzzle after completion"""
    name = 'tHeadPzlPostCompletion'
    column_prefix = 'i_tHeadPzlPostCompletion'
    @staticmethod
    def extract(subjects):
        def get_tHeadPzlPostCompletion(inst):
            sol, swap_l1, head_l1, last_swap, agg = [], [], [], 0.0, 0.0
            pzl_name = inst['CurrentPuzzle']['puzzleName']
            pzl_swap_list = inst['PiecesSwapList']
            in_head_list = inst['ObjectsCenterViewList']
            if in_head_list != []:
                for head in in_head_list:
                    head_len = head['length']
                    head_start = head['startTime']
                    head_name = head['name']
                    head_l1.append([head_start, head_name, head_len]) 
            if pzl_swap_list != []:
                for pzl in pzl_swap_list:
                    pzl_time = pzl['Time']
                    swap_l1.append([pzl_name, pzl_time]) 
                if swap_l1 != '' :
                    last_swap = swap_l1[-1]
                for g in head_l1:
                    if g[0] > last_swap[1] and ( g[1] == last_swap[0] or g[1][:5]=='Piece'):
                        agg += g[2]
            if agg > 0:
                sol = agg
            else:
                sol = np.NaN

            return sol
            sol.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleInstances', get_tHeadPzlPostCompletion , subjects)
        return df
PuzzlesInstanceTimeHeadPzlPostCompletion().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
id,,,,,,,,,,,,,,,,,,,,,
5BB9D23220AB8A117956A91C,NaN,0.838150,NaN,0.977234,2.425323,1.511902,1.162170,NaN,NaN,NaN,...,0.430237,0.200989,1.089539,2.227234,0.586548,1.837891,NaN,3.144592,1.321716,0.330994
5BBA2B3320AB8A117956A92D,0.412811,2.032806,1.183929,0.144989,0.279114,0.468781,0.625427,0.033051,0.971466,0.357361,...,8.488281,1.932556,0.056396,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5BBB4EB94AC518563CE8D571,0.694321,NaN,0.391464,0.149673,NaN,NaN,0.369812,2.769928,0.201355,1.189270,...,1.782715,2.014771,0.949951,1.631165,NaN,NaN,NaN,NaN,NaN,NaN
5BBB61C6DC69BC3FA50FA40A,1.608414,NaN,1.418472,NaN,0.055664,0.055420,1.754089,NaN,2.658661,NaN,...,0.938080,0.793579,2.122192,3.596100,2.624603,0.156616,0.64679,NaN,NaN,NaN


In [12]:
class PuzzleInstancePuzzleHead(BaseFeatureExtractor):
    """Total time head directed on the puzzle of the same instance"""
    name = 'puzzleHead'
    column_prefix = 'i_puzzleHead'
    @staticmethod
    def extract(subjects):
        def get_puzzleHead(inst):
            sol, head_l1, agg = [], [], 0.0
            pzl_name = inst['CurrentPuzzle']['puzzleName']
            in_head_list = inst['ObjectsCenterViewList']
            if in_head_list != []:
                for head in in_head_list:
                    head_len = head['length']
                    head_start = head['startTime']
                    head_name = head['name']
                    head_l1.append([head_start, head_name, head_len])

                for g in head_l1:
                    if g[1] == pzl_name or g[1][:5]=='Piece':
                        agg += g[2]
            if agg > 0:
                sol = agg
            else:
                sol = np.NaN        
            return sol
            sol.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleInstances', get_puzzleHead , subjects)
        return df
PuzzleInstancePuzzleHead().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
id,,,,,,,,,,,,,,,,,,,,,
5BB9D23220AB8A117956A91C,5.476410,5.625229,NaN,8.239609,37.876343,12.695679,2.726593,NaN,NaN,NaN,...,9.969849,4.891846,24.961243,7.024505,6.093872,13.080444,NaN,51.509949,16.069214,12.382141
5BBA2B3320AB8A117956A92D,8.712249,3.663544,8.265411,14.028320,3.194794,8.522858,0.625427,0.033051,13.068024,5.551300,...,17.680786,83.013123,12.421203,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5BBB4EB94AC518563CE8D571,2.838715,NaN,9.348923,4.561264,10.269989,6.674728,8.222534,4.010559,5.283173,6.539947,...,22.536316,9.492493,4.880798,58.121979,NaN,NaN,NaN,NaN,NaN,NaN
5BBB61C6DC69BC3FA50FA40A,13.392380,9.427078,1.418472,17.536514,9.829132,7.952484,6.713257,7.293701,7.349792,1.466125,...,5.204956,2.994293,19.524445,20.395630,9.081116,8.869202,62.638245,NaN,NaN,NaN


In [28]:
#Inserted
class PuzzleRoundPuzzleHead(BaseFeatureExtractor):
    """Total time head directed on the puzzle of the same instance"""
    name = 'r_tPuzzleHead'
    @staticmethod
    def extract(subjects):
        def tpuzzlehead(rnd):
            sol, head_l1, agg = [], [], 0.0
            pzl_name = rnd['CurrentPuzzle']['puzzleName']
            for inst in rnd['PuzzleInstances']:
                in_head_list = inst['ObjectsCenterViewList']
                if in_head_list:
                    for head in in_head_list:
                        head_len = head['length']
                        head_start = head['startTime']
                        head_name = head['name']
                        head_l1.append([head_start, head_name, head_len])

                    for g in head_l1:
                        if g[1] == pzl_name or g[1][:5]=='Piece':
                            agg += g[2]
                if agg > 0:
                    sol = agg
                else:
                    sol = np.NaN        
                return sol

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleRounds', tpuzzlehead , subjects)
        return df
PuzzleRoundPuzzleHead().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
id,,,,,,,,,,,,,,,,,,,,,
5C35A1D0B5997B32B4CEAA9B,NaN,3.485046,6.579224,9.583191,2.413269,8.690155,0.323914,NaN,6.657898,3.227844,...,19.68158,1.306763,2.412964,6.165619,7.918945,9.549927,8.432648,NaN,9.136841,3.97644


In [11]:
class PuzzlesRoundFPostVsPreCompletionDur(BaseFeatureExtractor):
    """tGazePostCompletion/tCompletion"""
    name = 'r_fPostVsPreCompletionDur'
    column_prefix = 'r_fPostVsPreCompletionDur'
    @staticmethod
    def extract(subjects_data):
        def get_r_fPostVsPreCompletionDur(subjects_data):
            ndf1 = get_dfc(subjects_data)
            ndf2 = get_dfg(subjects_data)
            ndf = ndf1 / ndf2
            return ndf
        df = get_r_fPostVsPreCompletionDur(subjects_data)
        return df
    
PuzzlesRoundFPostVsPreCompletionDur().extract(subjects_data)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
id,,,,,,,,,,,,,,,,,,,,,
5BB9D23220AB8A117956A91C,40.692888,183.176771,NaN,15.957103,247.048689,45.854133,11.529683,676.450609,2.114774,7.546546,...,3.171117,3.758979,13.981695,6.236224,4.278695,5.681751,3.964328,105.404098,11.247802,NaN
5BBA2B3320AB8A117956A92D,8.485492,0.957754,8.458139,12.916341,3.732605,5.476746,1.309071,21.582319,8.502235,15.169289,...,0.905823,37.400216,20.219131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5BBB4EB94AC518563CE8D571,6.594305,1.443028,9.893810,6.825199,7.318310,51.846712,1.927729,0.522800,5.323125,5.721186,...,13.765088,6.732269,4.696319,37.081048,NaN,NaN,NaN,NaN,NaN,NaN
5BBB61C6DC69BC3FA50FA40A,3.328434,2.708420,3.495421,75.453216,15.113092,687.266658,5.354319,4.631782,2.049027,1.756097,...,5.233502,0.957443,7.277119,6.115498,1.619468,3.649481,25.099544,NaN,NaN,NaN


In [29]:
#Inserted
class PuzzleTime1stGazeToClock(BaseFeatureExtractor):
    """time of first gaze to clock (from task onset)"""
    name = 't1stGazeToClock'
    column_prefix = 't_t1stGazeToClock'

    @staticmethod
    def extract(subjects):
        def get_t_t1stGazeToClock(subject):
            data_items = subject['Task_MultiPuzzles']['DataItems']
            start = data_items['TaskStartTime']
            for gaze in data_items['GazeAtObjectsList']:
                if gaze['name'].find('Timer') > -1:
                    return gaze['startTime'] - start
            return np.nan

        df = toolbox.to_dataframe(subjects_data, get_t_t1stGazeToClock)
        return df

PuzzleTime1stGazeToClock().extract(subjects_data)

,0
id,
5C35A1D0B5997B32B4CEAA9B,336.388879


In [30]:
#Inserted
class PuzzleTGazeToClock(BaseFeatureExtractor):
    """number of times gazed the clock during the entire task"""
    name = 'TGazeToClock'
    column_prefix = 't_nGazeToClock'
    @staticmethod
    def extract(subjects_data):
        def get_nGazeToClock(gaze):
            agg = 0.0
            if gaze['name'].find('Timer') > -1:
                agg += gaze['length']
            return agg
        df = toolbox.create_subjects_data_items_multi_dim_df('puzzles','GazeAtObjectsList', get_nGazeToClock, subjects_data)
        return df.sum(axis=1).to_frame()

PuzzleTGazeToClock().extract(subjects_data)

,0
id,
5C35A1D0B5997B32B4CEAA9B,0.333984


In [15]:
#Inserted
class PuzzlesTGazeIrrelevant(BaseFeatureExtractor):
    """time gazing at irrelevant things (i.e. walls, windows etc.), or eyes closed"""
    name = 'time_gaze_solution'
    column_prefix = 'r_tGazeIrrelevant'
    @staticmethod
    def extract(subjects):
        def get_TGazeIrrelevant(gaze):
            irr_tot = 0.0
            irrelevant_items = ['Plane Top Shadow', 'Tiles', 'Logo']            
            gaze_name = gaze['name']
            gaze_len = gaze['length']
            if gaze_name in irrelevant_items:
                irr_tot += gaze_len
                                   
            return irr_tot
            irr_tot.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'puzzles', 'GazeAtObjectsList', get_TGazeIrrelevant, subjects)
        #df.fillna(0, inplace=True)
        return df.sum(axis=1).to_frame()

PuzzlesTGazeIrrelevant().extract(subjects_data)

,0
id,
5BB9D23220AB8A117956A91C,36.587021
5BBA2B3320AB8A117956A92D,6.838135
5BBB4EB94AC518563CE8D571,5.272034
5BBB61C6DC69BC3FA50FA40A,8.169876


In [16]:
#Inserted
class PuzzlesTaskNumHeadToClock(BaseFeatureExtractor):
    """number of times directed head at the clock during the entire task"""
    name = 'r_get_t_nHeadToClock'
    column_prefix = 't_nHeadToClock'
    @staticmethod
    def extract(subjects_data):
        def get_t_nHeadToClock(head):
            clock_head = 0
            if head['name'].find('Timer') > -1:
                clock_head += 1

            return clock_head
            clock_head.append()

        df = toolbox.create_subjects_data_items_multi_dim_df(
        'puzzles', 'CenterViewObjectsList', get_t_nHeadToClock, subjects_data)
        #df.fillna(0, inplace=True)
        return df.sum(axis=1).to_frame()

PuzzlesTaskNumHeadToClock().extract(subjects_data)

,0
id,
5BB9D23220AB8A117956A91C,1.0
5BBA2B3320AB8A117956A92D,2.0
5BBB4EB94AC518563CE8D571,2.0
5BBB61C6DC69BC3FA50FA40A,0.0


In [17]:
#Inserted
class PuzzleTime1stHeadToClock(BaseFeatureExtractor):
    """time of first head to clock (from task onset)"""
    name = 't1stHeadToClock'
    column_prefix = 't_t1stHeadToClock'
    
    @staticmethod
    def extract(subjects):
        def get_t_t1stHeadToClock(subject):
            data_items = subject['Task_MultiPuzzles']['DataItems']
            start = data_items['TaskStartTime']
            for head in data_items['CenterViewObjectsList']:
                if head['name'] == 'TimerPanel':
                    return head['startTime'] - start
            return np.nan

        df = toolbox.to_dataframe(subjects_data, get_t_t1stHeadToClock)
        return df

PuzzleTime1stHeadToClock().extract(subjects_data)

,0
id,
5BB9D23220AB8A117956A91C,NaN
5BBA2B3320AB8A117956A92D,NaN
5BBB4EB94AC518563CE8D571,NaN
5BBB61C6DC69BC3FA50FA40A,NaN


In [36]:
#Inserted
class PuzzleTimeatBtwGazeToClock(BaseFeatureExtractor):
    """duration between consecutive gazes to clock during the entire task (list).
        First duration is from task onset to first gaze to clock."""
    name = 't_atBtwGazeToClock'
    @staticmethod
    def extract(subjects):
        def get_t_atBtwGazeToClock(subject):
            gtime, sol, gc = [], [], 0
            data_items = subject['Task_MultiPuzzles']['DataItems']
            start = data_items['TaskStartTime']
            gaze_list = data_items['GazeAtObjectsList']
            for gz1, gz2 in zip(gaze_list, gaze_list[1:]):
                if gz1['name'].find('Timer') > -1 and gz2['name'].find('Timer') < 0:
                    gtime.append(gz1['startTime'] - start)
                    gc += 1
            if gc > 1:    
                for g1, g2 in zip(gtime, gtime[1:]):
                    delta = g2 - g1
                    sol.append(delta)
            elif gc == 1:
                sol = gtime
            else:
                sol = []
            return sol
        df = toolbox.to_dataframe(subjects_data, get_t_atBtwGazeToClock)
        return df

PuzzleTimeatBtwGazeToClock().extract(subjects_data)

,0
id,
5C35A1D0B5997B32B4CEAA9B,1.159156


In [6]:
#Inserted
class PuzzleNumAtBtwGazeToClock(BaseFeatureExtractor):
    """count consecutive gazes to clock during the entire task (list).
        First duration is from task onset to first gaze to clock."""
    name = 't_natBtwGazeToClock'
    @staticmethod
    def extract(subjects):
        def tanbtwgazetoclock(subject):
            gtime, sol, gc = [], [], 0
            task = subject.get('Task_MultiPuzzles', [])
            if not task['DataItems']:
                return []
            data_items = task['DataItems']
            if not data_items['GazeAtObjectsList']:
                return []
            gaze_list = data_items['GazeAtObjectsList']
            for g1, g2 in zip(gaze_list, gaze_list[1:]):
                if g1['name'].find('Timer') > -1 and g2['name'].find('Timer') < 0:
                    gc += 1
            return gc
        df = toolbox.to_dataframe(subjects_data, tanbtwgazetoclock)
        return df

PuzzleNumAtBtwGazeToClock().extract(subjects_data)

,0
id,
5C35A1D0B5997B32B4CEAA9B,2


In [40]:
#Inserted
class PuzzleNumatBtwHeadToClock(BaseFeatureExtractor):
    """count consecutive head direction to clock during the entire task (list).
    First duration is from task onset to first gaze to clock."""
    name = 'n_atBtwHeadToClock'
    
    @staticmethod
    def extract(subjects):
        def nanbtwgazetoclock(subject):
            gtime, sol, gc = [], [], 0
            data_items = subject['Task_MultiPuzzles']['DataItems']
            head_list = data_items['CenterViewObjectsList']
            for g1, g2 in zip(head_list, head_list[1:]):
                if g1['name'].find('Timer') > -1 and g2['name'].find('Timer') < 0:
                    gc += 1                  
            return gc

        df = toolbox.to_dataframe(subjects_data, nanbtwgazetoclock)
        return df

PuzzleNumatBtwHeadToClock().extract(subjects_data)

,0
id,
5C35A1D0B5997B32B4CEAA9B,0


In [44]:
#Inserted
class PuzzleTimeatBtwHeadToClock(BaseFeatureExtractor):
    """duration between consecutive gazes to clock during the entire task (list).
        First duration is from task onset to first gaze to clock."""
    name = 't_atBtwHeadToClock'
    @staticmethod
    def extract(subjects):
        def get_t_atBtwHeadToClock(subject):
            gtime, sol, gc = [], [], 0
            data_items = subject['Task_MultiPuzzles']['DataItems']
            start = data_items['TaskStartTime']
            head_list = data_items['CenterViewObjectsList']
            for gz1, gz2 in zip(head_list, head_list[1:]):
                if gz1['name'].find('Timer') > -1 and gz2['name'].find('Timer') < 0:
                    gtime.append(gz1['startTime'] - start)
                    gc += 1
            if gc > 1:    
                for g1, g2 in zip(gtime, gtime[1:]):
                    delta = g2 - g1
                    sol.append(delta)
            elif gc == 1:
                sol = gtime
            else:
                sol = []
            return sol
        df = toolbox.to_dataframe(subjects_data, get_t_atBtwHeadToClock)
        return df

PuzzleTimeatBtwHeadToClock().extract(subjects_data)

""
id
5C35A1D0B5997B32B4CEAA9B


In [46]:
#Inserted
class PuzzleTimeNumPzlLeftUnsolved(BaseFeatureExtractor):
    """number of puzzles started (i.e. at leas 1 swap made), but left unsolved"""
    name = 't_nPzlLeftUnsolved'
    @staticmethod
    def extract(subjects):
        def get_t_nPzlLeftUnsolved(rnd):
            pzl_count = 0
            if rnd['PuzzleUserAnswer'] != None and rnd['PiecesSwapList'] != None and rnd['IsCorrect'] == 'false':
                pzl_count += 1            
            return pzl_count

        df = toolbox.create_subjects_data_items_multi_dim_df(
            'puzzles','PuzzleRounds', get_t_nPzlLeftUnsolved , subjects)
        return df.sum(axis=1).to_frame()
PuzzleTimeNumPzlLeftUnsolved().extract(subjects_data)

,0
id,
5C35A1D0B5997B32B4CEAA9B,0
